<h3>Parameters used in magMasks:</h3>
<ul>
	<li>pathImagemMag: the computer path where the Magnetogram image will be loaded from</li>
    <li>bw_mask: the matrix of ?????????????????????????</li>
</ul>

<h3>Functions used in magMasks:</h3>
<ul>
    <li>calc_mu_hmi(thresh)</li>
</ul>

#### 1) Importing a file with all the library needed on the code:

In [1]:
from funcoes import *

Using TensorFlow backend.


#### 2) Importing the defined values in settings.py. They are url, path, dataInicial, dataFinal, resolucao and sufixo.


In [3]:
import settings

#### 3) Importing the Input variables needed for this functions, that were saved in the preivous code (imagesMasks.py):

In [ ]:
filename = path + 'IOFiles\\' + 'inputMagMasks.out'

my_shelf = shelve.open(filename)

for key in my_shelf:
    globals()[key]=my_shelf[key]

my_shelf.close()

#### 4) Loading the magnetogram image:

In [4]:
try:
    imagemDaVez = cv2.imread(pathImagemMag)

except:
    print("magMasks: Problemas ao carregar a imagem " + pathImagemMag + "!")

NameError: name 'pathImagemMag' is not defined

#### 5) Converting the image to gray scale and cleaning its background.

In [3]:
try:    
    imagemCinza = cv2.cvtColor(imagemDaVez, cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imagemCinza,10,1,cv2.THRESH_BINARY)

    '''MU_RINGS ESTÁ VINDO SEM NAN, DIFERENTE DO QUE ACONTECE NO MATLAB, O QUE ESTÁ DESTOANDO OS VALORES CALCULADOS PARA O BW_MASK, EM RELAÇÃO AOS CALCULADOS NO MATLAB'''
    mu, mu_rings = calc_mu_hmi(thresh)

    area_disk = np.count_nonzero(mu>0)

except:
    print("magMasks: Erro nas etapas de limpeza e de definição do disco solar e dos anéis" + pathImagemMag)  

NameError: name 'pathImagemMag' is not defined

In [ ]:
try:
    bw1 = imagemCinza>=(128+20)
    bw2 = imagemCinza<=(128-20)
    bw3 = bw1 | bw2
    bw4 = np.uint8(bw3 & (mu_rings > 0))

    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(bw4, connectivity=4)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    min_size = 10 
    bw5 = np.zeros((output.shape))

    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            bw5[output == i + 1] = 1

    labels = label(bw5, connectivity=1)

    props = regionprops(labels)

    area = []

    for region in props:
        area.append(1e6*region.area/area_disk)
    #area_total.append(area) #apenas uma verificação das áreas totais calculadas

    '''VERIFICAR SE NA LINHA ABAIXO DEVE REALMENTE SER EXCLUÍDA A PRIMEIRA ÁREA (VALOR MÁXIMO), CUJO VALOR NÃO CONDIZ COM OS RESULTADOS EM MATLAB'''
    area = np.asarray(area[1:])
    props = props[1:]
except:
    print("magMasks: Erro na etapa de pré classificação da imagem " + pathImagemMag)  

In [ ]:
try:  
    '''ESSES VALORES DE THRESHOLDS DEVEM FICAR EM UM ARQUIVO MESMO?? COMO SÃO ESTIMADOS???'''
    th1_m = float(16.6883)
    th2_m = float(24.8066)
    th3_m = float(89.0673)

    bw_small = np.zeros(bw5.shape)
    ii = np.where(area <= th1_m)
    ii = np.array(ii)[0]
    flag_small = 0        
    if ii.size > 0:
        flag_small = 1
        for i in range (ii.size):
            x = props[ii[i]]['coords'][:,0]
            y = props[ii[i]]['coords'][:,1]
            bw_small[x,y] = 1 

    bw_media1 = np.zeros(bw5.shape)            
    ii = np.where((area > th1_m) & (area <= th2_m))
    ii = np.array(ii)[0]
    flag_media1 = 0        
    if ii.size > 0:
        flag_media1 = 1
        for i in range (ii.size):
            x = props[ii[i]]['coords'][:,0]
            y = props[ii[i]]['coords'][:,1]
            bw_media1[x,y] = 1 

    bw_media2 = np.zeros(bw5.shape)            
    ii = np.where((area > th2_m) & (area <= th3_m))
    ii = np.array(ii)[0]
    flag_media2 = 0        
    if ii.size > 0:
        flag_media2 = 1
        for i in range (ii.size):
            x = props[ii[i]]['coords'][:,0]
            y = props[ii[i]]['coords'][:,1]
            bw_media2[x,y] = 1 

    bw_large = np.zeros(bw5.shape)            
    ii = np.where(area > th3_m)
    ii = np.array(ii)[0]
    flag_large = 0        
    if ii.size > 0:
        flag_large = 1
        for i in range (ii.size):
            x = props[ii[i]]['coords'][:,0]
            y = props[ii[i]]['coords'][:,1]
            bw_large[x,y] = 1 



    bw_mask[np.where(mu_rings > 0)] = 1               

    if flag_small:
        bw_mask[np.where(bw_small==1)] = 2
    if flag_media1:
        bw_mask[np.where(bw_media1==1)] = 3
    if flag_media2:
        bw_mask[np.where(bw_media2==1)] = 4
    if flag_large:
        bw_mask[np.where(bw_large==1)] = 5

except:
    print("magMasks: Erro no processo de classificação da imagem " + pathImagemMag)